# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение, количество детей клиента, уровень дохода и цели кредита на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Цель исследования — проверить четыре гипотезы:

**1. Количество детей у клиента влияет на факт погашения кредита в срок. Клиенты, у которых детей больше, реже гасят кредиты в срок.**  

**2. Семейное положение влияет на факт погашения кредита в срок. Клиенты, состоящие в браке, чаще гасят кредиты в срок по сравнению с теми, кто никогда не состоял в браке.** 

**3. Уровень дохода влияет на факт погашения кредита в срок. Чем выше уровень дохода клиентов, тем чаще они гасят кредит в срок.**  

**4. Цели кредита влияют на факт погашения кредита в срок. Клиенты, которые берут кредиты на покупку автомобиля, реже гасят их в срок.** 

Данные клиентах банка мы получили из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы поищем возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

`Обзор данных.`  
`Предобработка данных.`  
`Проверка гипотез.`

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

Составляем первое представление о данных банка

Импортируем библиотеку `pandas`.

In [10]:
import pandas as pd # импорт библиотеки pandas

Читаем файл data.csv и сохраняем его в переменной df:

In [11]:
df = pd.read_csv('C:\\Users\\admin\\credit_scoring_research_project.csv') # чтение файла с данными и сохранение в df

Выведим на экран первые десять строк таблицы: 

In [12]:
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получаем общую информацию о таблице:

In [13]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. Тип данных в пяти столбцах — `object`, в пяти столбцах - `int` и в двух столбцах - `float`.

Согласно документации к данным:

`children` — количество детей в семье  
`days_employed` — общий трудовой стаж в днях  
`dob_years` — возраст клиента в годах  
`education` — уровень образования клиента  
`education_id` — идентификатор уровня образования  
`family_status` — семейное положение  
`family_status_id` — идентификатор семейного положения  
`gender` — пол клиента  
`income_type` — тип занятости  
`debt` — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход в рублях   
`purpose` — цель получения кредита

В названиях колонок нарушений стиля нет.  
В столбце `education` значения записаны в разных регистрах. Для текущего исследования эти данные не представляют интереса, однако, для избежания возможных скрытых дубликатов лучше привести значения к единому регистру.   
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Подробно изучим колонки с интересующими нас данными:

In [14]:
df['family_status'].value_counts() # подсчет уникальных значений в колонке family_status

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В колонке `family_status` имеется пять вариантов семейного статуса. Больше всего клиентов, состоящих в браке.

In [15]:
df['family_status_id'].value_counts() # подсчет уникальных значений в колонке family_status_id

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

В колонке `family_status_id` также имеется пять вариантов семейного статуса, которые обозначены от 0 до 4 и соответствуют статусам из колонки `family_status`.

In [16]:
df['debt'].value_counts() # подсчет уникальных значений в колонке debt

0    19784
1     1741
Name: debt, dtype: int64

В колонке `debt` имеется всего два обозначения:  
0 - нет задолженности  
1 - есть задолженность

In [17]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонке `children` указано количество детей клиента, однако есть неправдоподобные данные: не может быть "минус один" ребенок. Также невозможно, чтобы у 76 клиентов было по двадцать детей, при том, что у остальных не больше пяти.  
Эти ошибки, наиболее вероятно, связаны с неправильным внесением данных.  
"-1" должно соответствовать "1", а "20" должно соответствовать "2".

In [18]:
df['purpose'].value_counts() # подсчет уникальных значений в колонке purpose

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [19]:
len(df['purpose'].value_counts()) # подсчет длины списка уникальных значений в колонке debt

38

В колонке `purpose` имеется 38 уникальных целей кредита. В дальнейшем эти данные будет необходимо уточнить с помощью лемматизации.

**Вывод**

При изучении датасета мы определили, что необходимые для нашего исследования данные содержатся в следующих колонках:   
`children` — количество детей в семье  
`family_status` — семейное положение  
`debt` — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход  
`purpose` — цель получения кредита

Предварительно можно утверждать, что, данных достаточно для проверки гипотез, однако, мы выявили следующие проблемы в данных, которые будет необходимо устранить на этапе предобработки:  
1. В колонке `children` есть неявные дубликаты.
2. В колонке `total_income` есть пропущенные значения.

Также на этапе предобработки будет необходимо произвести поиск и удаление явных дубликатов.

## Предобработка данных

### Обработка пропусков  

Сначала посчитаем, сколько в таблице пропущенных значений:

In [20]:
df.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски имеются в двух колонках: `days_employed` и `total_income`.

Для того, чтобы заменить или восстановить пропущенные значения, детально изучим эти колонки. 

In [21]:
df['days_employed'].head(10) # получение первых 10 значений колонки days_employed

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

Мы видим, что в колонке имеются отрицательные значения, а также неправдоподобно большие значения, например, в пятой строке трудовой стаж составляет больше 340 тысяч дней!  

Предположим, что отрицательные значения появились из-за ошибки при внесении данных и приведем все значения в колонке к абсолютным, а затем посмотрим диапазон значений.

In [22]:
df['days_employed'] = abs(df['days_employed']) # приведение значений колонки days_employed к абсолютным

In [23]:
df['days_employed'].min() # получение минимального значения 

24.14163324048118

In [24]:
df['days_employed'].max() # получение макимального значения

401755.40047533

Получается, что минимальный трудовой стаж составил 24 дня, а максимальный больше 1000 лет!

Теперь изучим колонку `total_income`.

In [25]:
df['total_income'].head(10) # получение первых 10 значений колонки total_income

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
5    255763.565419
6    240525.971920
7    135823.934197
8     95856.832424
9    144425.938277
Name: total_income, dtype: float64

Изучим диапазон значений в колонке.

In [26]:
df['total_income'].max() # получение макимального значения

2265604.028722744

In [27]:
df['total_income'].min() # получение минимального значения 

20667.26379327158

In [28]:
df['total_income'].mean() # получение среднего значения

167422.30220817294

In [29]:
df['total_income'].median() # получение медианы

145017.93753253992

Ежемесячный доход клиентов находится в диапазоне от ~20 тысяч до ~2 миллионов рублей в месяц. Средний и медианных доход различаются не сильно. Таким образом, мы считаем, что данные указаны корректно, а пропуски в данных связаны с ошибками при их внесении.

Заполняем пропущенные значения в колонке `total_income` медианными.

In [30]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median()) # заполнение пропущенных значений

В колонке `days_employed` указано слишком много некорректных данных. Так как значения этой колонки не представляют интереса для проведения нашего исследования, то целесообразно ее удалить.

In [31]:
df.dropna(axis = 'columns', inplace = True) # удаление колонок с пропущенными значениями

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [32]:
df.isna().sum() # подсчёт пропусков

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски имелись в двух колонках: `days_employed` и `total_income`. В колонке `total_income` мы заполнили пропуски медианными значениями ежемесячного дохода, а колонку `days_employed` удалили, так как в значениях было слишком много ошибок, а ценности для нашего исследования она не представляла.

### Замена типа данных

Ранее при изучении колонки `total_income` мы определили, что значения в ней указаны в виде вещественных чисел. Для большей наглядности и удобства операций с этими значениями переведем значения колонки в целочисленные.

In [33]:
df['total_income'] = df['total_income'].astype('int') # преобразование значений колонки total_income в целочисленные

### Обработка дубликатов

Посчитаем явные дубликаты в таблице, но перед этим приведем значения колонки `education` к нижнему регистру:

In [34]:
df['education'] = df['education'].str.lower() # перевод значений колонки education в нижний регистр

In [35]:
f'Количество дубликатов {df.duplicated().sum()}' # подсчёт явных дубликатов

'Количество дубликатов 71'

Вызовем специальный метод pandas, чтобы удалить явные дубликаты:

In [36]:
df = df.drop_duplicates().reset_index(drop = True) # удаление явных дубликатов с удалением старых индексов и формированием новых

Проверим, что дубликаты устранены:

In [37]:
f'Количество дубликатов {df.duplicated().sum()}' # подсчёт явных дубликатов

'Количество дубликатов 0'

Ранее мы обнаружили в колонке `children` неявные дубликаты: -1 вместо 1 и 20 вместо 2.  
Для того, чтобы избавиться от дубликатов, приведем значения колонки к абсолютным и заменим 20 на 2 с помощью специального метода pandas:

In [38]:
df['children'] = abs(df['children']) # приведение значений колонки children к абсолютным

In [39]:
df['children'] = df['children'].replace(20, 2) # устранение неявного дубликата

Проверим, что дубликаты устранены:

In [40]:
df['children'].value_counts() # подсчет уникальных значений колонки children

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

Мы обнаружили 71 явный дубликат в датасете и 2 варианта неявных дубликатов в колонке `children` и устранили их.

### Лемматизация

Уточним уникальные цели кредита с помощью лемматизации.  
Для этого импортируем из библиотеки pymystem3 функцию леммаизации Mystem.

In [33]:
from pymystem3 import Mystem # импорт функции лемматизации Mystem
m = Mystem()

Создаем series с уникальными значениями целей кредита.

In [34]:
purpose_list = df['purpose'].unique() # сохранение series со значениями целей кредита в переменную

Создаем пустой список для сохранения в него лемматизированных целей кредита и заполняем его с помощью цикла.

In [35]:
lemmas = [] # создание пустого списка

In [36]:
# каждую строчку в 'purpose_list' лемматизируем, получаем список лемм и добавляем значения в пустой список 'lemmas'
for purpose in purpose_list:
    lemma = m.lemmatize(purpose)
    lemmas.append(lemma)

In [37]:
lemmas # выводим список на экран

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Вывод**

По результатам лемматизации мы определили, что все цели кредита можно свести к четырем категориям: операции с недвижимостью, покупка автомобиля, образование и расходы на свадьбу.

### Категоризация данных

Как мы определили выше, для проверки гипотез нам необходимы данные из колонок `children`, `family_status`, `debt`,`total_income` и `purpose`, однако, не все данные представлены в удобном для анализа виде.  
Если в колонках `children` и  `family_status` имеется всего по пять уникальных значений, то колонке `total_income` уникальных значений очень много, а вколонке purpose имеется 38 вариантов формулировки целей, хотя, как мы уже установили, их всего четыре. В этих случаях необходимо произвести категоризацию данных.  
Также, для большей наглядности и удобства построения сводных таблиц, мы приняли решение произвести категоризацию данных из колонки `debt`.

Создадим новую колонку для категорий целей кредита. Для этого создадим функцию, которая разобьет цели по четырем категориям, а затем, для проверки результата подсчитаем количество значений в каждой категории.

In [41]:
# разбиваем на категории по леммам: "Жилье/недвижимость", "автомобиль", "образование", "свадьба" 
def purpose_to_category(purpose):
    if 'авто' in purpose:
        return 'автомобиль'
    elif 'свад' in purpose:
        return  'свадьба'
    elif 'образован' in purpose:
        return 'образование'
    elif 'недвиж' or 'жил' in purpose:
        return  'недвижимость'

In [42]:
df['purpose_category']= df['purpose'].apply(purpose_to_category) # создаем новую колонку для категорий

In [43]:
df['purpose_category'].value_counts() # проверяем результат категоризации

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Создадим новую колонку для категорий ежемесячного дохода.  
Ранее мы установили, что ежемесячный доход клиентов находится в диапазоне от ~20 тысяч до ~2 миллионов рублей в месяц. Средний и медианных доход различаются не сильно и составляют ~167 тысяч и ~145 тысяч рублей соответственно.   
На основании этих данных мы определили четыре категории ежемесячного дохода: "низкий доход", "средний доход", "высокий доход" и "очень высокий доход".  
Создадим функцию, которая разобьет цели по этим категориям, а затем, для проверки результата подсчитаем количество значений в каждой категории.

In [44]:
# разбиваем ежемесячный доход на категории
def total_income_to_category(total_income):
    if total_income <= 75000:
        return 'низкий доход'
    if 75001 <= total_income <= 150000:
        return 'средний доход'
    elif 150001 <= total_income <= 500000:
        return  'высокий доход'
    elif 500001 <= total_income:
        return 'очень высокий доход'

In [45]:
df['total_income_category']= df['total_income'].apply(total_income_to_category) # создаем новую колонку для категорий

In [46]:
df['total_income_category'].value_counts() # проверяем результат категоризации

средний доход          10405
высокий доход           8962
низкий доход            1865
очень высокий доход      222
Name: total_income_category, dtype: int64

Создадим новую колонку для категорий по факту наличия или отсутствия задолженности клиентов перед банком: 'должник', если клиент не вернул кредит вовремя, и 'вернул кредит'.

In [47]:
# разбиваем колонку debt на категории
def debt_to_category(debt):
    if debt > 0:
        return 'должник'
    else:
        return 'вернул кредит'

In [48]:
df['debt_category'] = df['debt'].apply(debt_to_category) # создаем новую колонку для категорий

Проверим результаты предобработки данных:

In [49]:
df.head(10) # получение первых 10 строк таблицы df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category,debt_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий доход,вернул кредит
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход,вернул кредит
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний доход,вернул кредит
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход,вернул кредит
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий доход,вернул кредит
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий доход,вернул кредит
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий доход,вернул кредит
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний доход,вернул кредит
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний доход,вернул кредит
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,средний доход,вернул кредит


In [50]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
children                 21454 non-null int64
dob_years                21454 non-null int64
education                21454 non-null object
education_id             21454 non-null int64
family_status            21454 non-null object
family_status_id         21454 non-null int64
gender                   21454 non-null object
income_type              21454 non-null object
debt                     21454 non-null int64
total_income             21454 non-null int32
purpose                  21454 non-null object
purpose_category         21454 non-null object
total_income_category    21454 non-null object
debt_category            21454 non-null object
dtypes: int32(1), int64(5), object(8)
memory usage: 2.2+ MB


**Вывод**

На этапе **предобработки данных** мы заполнили пропущенные значения, удалили явные дубликаты, избавились от неявных дубликатов, привели значения к правильным типам, произвели лемматизацию и категоризацию данных.  
Можно утверждать, что данные готовы для проверки гипотез.

## Проверка гипотез

### Проверка первой гипотезы

Первая гипотеза утверждает, что количество детей у клиента влияет на факт погашения кредита в срок, при этом клиенты, у которых детей больше, реже гасят кредиты в срок.  
Проверим это предположение по данным о количестве должников среди клиентов с разным количеством детей.  
Для этого:
* создадим сводную таблицу количества должников среди клиентов с разным количеством детей
* посчитаем отношение должников к общему количеству клиентов
* отсортируем получившееся значение по убыванию

In [51]:
# создаем сводную таблицу
df_pivot_debt_to_children = df.pivot_table(index = 'children', columns = 'debt_category', values = 'debt', aggfunc = 'count')

In [52]:
# считаем отношение должников ко всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_to_children['ratio'] = df_pivot_debt_to_children['должник'] / (df_pivot_debt_to_children['должник'] + df_pivot_debt_to_children['вернул кредит']) * 100

In [53]:
# сортируем таблицу по колонке ratio
df_pivot_debt_to_children.sort_values('ratio', ascending = False)

debt_category,вернул кредит,должник,ratio
children,,,
4,37.0,4.0,9.756098
2,1926.0,202.0,9.492481
1,4410.0,445.0,9.165808
3,303.0,27.0,8.181818
0,13028.0,1063.0,7.543822
5,9.0,NaN,NaN


**Вывод**

Наибольшое количество должников присутствует среди людей с четырьмя детьми. В то же время количество должников среди клиентов с тремя детьми меньше, чем среди клиентов с двумя детьми и одним ребенком, а среди людей с пятью детьми должников вообще нет.  
Таким образом, количество детей не влияет на факт погашения кредита в срок. Первая гипотеза не подтвердилась.  
Это вполне объяснимо, так как многодетные семьи получают льготы по кредитам и другие льготы, которые снижают нагрузку на кошелек. Также списание долгов у них происходит по упрощенной схеме, а уволить таких людей с работы практически невозможно. Не стоит забывать и про материнский капитал.

### Проверка второй гипотезы

Вторая гипотеза утверждает, что семейное положение клиента влияет на факт погашения кредита в срок, при этом клиенты, состоящие в браке, чаще гасят кредиты в срок.  
Проверим это предположение по данным о количестве должников среди клиентов с разным семейным положением.  
Для этого:
* создадим сводную таблицу количества должников среди клиентов с разным семейным положением
* посчитаем отношение должников к общему количеству клиентов
* отсортируем получившееся значение по убыванию


In [54]:
# создаем сводную таблицу
df_pivot_debt_to_family_status = df.pivot_table(index = 'family_status', columns = 'debt_category', values = 'debt', aggfunc = 'count')

In [55]:
# считаем отношение должников ко всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_to_family_status['ratio'] = df_pivot_debt_to_family_status['должник'] / (df_pivot_debt_to_family_status['должник'] + df_pivot_debt_to_family_status['вернул кредит']) * 100

In [56]:
# сортируем таблицу по колонке ratio
df_pivot_debt_to_family_status.sort_values('ratio', ascending = False)

debt_category,вернул кредит,должник,ratio
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


**Вывод**

Наибольшее количество должников присутствует среди людей, которые никогда не состояли в официальном браке. В то же время мешьше всего должников среди клиентов, которые состоят или ранее состояли в браке.  
Таким образом, семейное положение клиента влияет на факт погашения кредита в срок, клиенты, состоящие в браке, чаще гасят кредиты в срок по сравнению с теми, кто никогда не состоял в браке. Вторая гипотеза подтвердилась полностью.  
Это также вполне объяснимо, так как в браке финансовая нагрузка распределяется между супругами и, в случае потери дохода одним из супругов, кредит может быть погашен за счет доходов другого супруга.

### Проверка третьей гипотезы

Третья гипотеза утверждает, что уровень дохода клиента влияет на факт погашения кредита в срок, при этом чем выше уровень дохода клиентов, тем чаще они гасят кредиты в срок.  
Проверим это предположение по данным о количестве должников среди клиентов с разным уровне дохода.
Для этого:

* создадим сводную таблицу количества должников среди клиентов с разным уровнем дохода
* посчитаем отношение должников к общему количеству клиентов
* отсортируем получившееся значение по убыванию

In [57]:
# создаем сводную таблицу
df_pivot_debt_to_total_income_category = df.pivot_table(index = 'total_income_category', columns = 'debt_category', values = 'debt', aggfunc = 'count')

In [58]:
# считаем отношение должников к всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_to_total_income_category['ratio'] = df_pivot_debt_to_total_income_category['должник'] / (df_pivot_debt_to_total_income_category['должник'] + df_pivot_debt_to_total_income_category['вернул кредит']) * 100

In [59]:
# сортируем таблицу по колонке ratio
df_pivot_debt_to_total_income_category.sort_values('ratio', ascending = False)

debt_category,вернул кредит,должник,ratio
total_income_category,,,
средний доход,9526,879,8.447862
высокий доход,8250,712,7.944655
низкий доход,1729,136,7.292225
очень высокий доход,208,14,6.306306


**Вывод**

Наименьшее количество должников присутствует среди людей с очень высоким уровнем дохода, в то же время должников среди людей с низким доходом тоже мало, что, однако, может говорить о тщательной проверке этих людей перед выдачей кредита. Тем не менее, на основе имеющихся данных однозначно судить о том, что уроень дохода клиента влияет на факт погашения кредита в срок нельзя.  
Таким образом, третья гипотеза не подтвердилась.  
Это можно объяснить, по аналогии с многодетными семьями, тем, что малоимущие клиенты в случае финансовых затруднений раз в несколько лет могут получить поддержку от государства в виде субсидий или списания долгов.

### Проверка четвертой гипотезы

Согласно четвертой гипотезе, цель кредита влияет на факт погашения кредита в срок, а клиенты, которые берут кредиты на покупку автомобиля, реже гасят их в срок.  
Проверим это предположение по данным о количестве должников среди клиентов с разными целями кредита.
Для этого:

* создадим сводную таблицу количества должников среди клиентов с разными целями кредита
* посчитаем отношение должников к общему количеству клиентов
* отсортируем получившееся значение по убыванию

In [60]:
# создаем сводную таблицу
df_pivot_debt_to_purpose_category = df.pivot_table(index = 'purpose_category', columns = 'debt_category', values = 'debt', aggfunc = 'count')

In [61]:
# считаем отношение должников ко всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_to_purpose_category['ratio'] = df_pivot_debt_to_purpose_category['должник'] / (df_pivot_debt_to_purpose_category['должник'] + df_pivot_debt_to_purpose_category['вернул кредит']) * 100

In [62]:
# сортируем таблицу по колонке ratio
df_pivot_debt_to_purpose_category.sort_values('ratio', ascending = False)

debt_category,вернул кредит,должник,ratio
purpose_category,,,
автомобиль,3903,403,9.359034
образование,3643,370,9.220035
свадьба,2138,186,8.003442
недвижимость,10029,782,7.233373


**Вывод**

Наибольшее количество должников присутствует среди клиентов, которые взяли кредит для покупки автомобиля и на образование, а наименьшее среди взявших кредит на приобретение недвижимости.  
Таким образом, цель кредита влияет на факт погашения кредита в срок. Гипотеза подтвердилась.  
Это может быть связано с тем, что приобретенная недвижимость может приносить владельцу дополнительный доход за счет сдачи в аренду, а в случае финансовых затруднений перейти к банку в счет погашения кредита. К тому же, согласно тренду, цены на недвижимость растут, что делает выгодной дальнейшую перепродажу. В то же время автомобиль после покупки сразу сильно теряет в цене, а также требует постоянных расходов на топливо, обслуживание и т.д.

## Итоги исследования

Мы проверили четыре гипотезы и установили:

**1. Количество детей у клиента не влияет на факт погашения кредита в срок.**

Наибольшое количество должников присутствует среди людей с четырьмя детьми. В то же время количество должников среди клиентов с тремя детьми меньше, чем среди клиентов с двумя детьми и одним ребенком, а среди людей с пятью детьми должников вообще нет.  
**Первая гипотеза не подтвердилась**.   
Это вполне объяснимо, так как многодетные семьи получают льготы по кредитам и другие льготы, которые снижают нагрузку на кошелек. Также списание долгов у них происходит по упрощенной схеме, а уволить таких людей с работы практически невозможно. Не стоит забывать и про материнский капитал.

**2. Семейное положение клиента влияет на факт погашения кредита в срок. Клиенты, состоящие в браке, чаще гасят кредиты в срок по сравнению с теми, кто никогда не состоял в браке.**

Наибольшее количество должников присутствует среди людей, которые никогда не состояли в официальном браке. В то же время мешьше всего должников среди клиентов, которые состоят или ранее состояли в браке.   
**Вторая гипотеза полностью подтвердилась.**  
Это также вполне объяснимо, так как в браке финансовая нагрузка распределяется между супругами и, в случае потери дохода одним из супругов, кредит может быть погашен за счет доходов другого супруга.

**3. Уровень дохода не влияет на факт погашения кредита в срок.**  
Наименьшее количество должников присутствует среди людей с очень высоким уровнем дохода, в то же время должников среди людей с низким доходом тоже мало, что, однако, может говорить о тщательной проверке этих людей перед выдачей кредита. Тем не менее, на основе имеющихся данных однозначно судить о том, что уроень дохода клиента влияет на факт погашения кредита в срок нельзя.  
**Третья гипотеза не подтвердилась.**  
Это можно объяснить, по аналогии с многодетными семьями, тем, что малоимущие клиенты в случае финансовых затруднений раз в несколько лет могут получить поддержку от государства в виде субсидий или списания долгов.

**4. Цели кредита влияют на факт погашения кредита в срок. Клиенты, которые берут кредиты на покупку автомобиля, реже гасят их в срок.**  
Наибольшее количество должников присутствует среди клиентов, которые взяли кредит для покупки автомобиля и на образование, а наименьшее среди взявших кредит на приобретение недвижимости.  
**Четвертая гипотеза полностью подтвердилась.**  
Это наиболее вероятно связано с тем, что приобретенная недвижимость может приносить владельцу дополнительный доход за счет сдачи в аренду, а в случае финансовых затруднений перейти к банку в счет погашения кредита. К тому же, согласно тренду, цены на недвижимость растут, что делает выгодной дальнейшую перепродажу. В то же время автомобиль после покупки сразу сильно теряет в цене, а также требует постоянных расходов на топливо, обслуживание и т.д.